# Section 2.1: Review of Linear Algebra and Matrix Theory Exercises

These exercises will help you strengthen your understanding of fundamental linear algebra concepts that are essential for complex systems modeling.

## Required Libraries

We'll use NumPy for linear algebra operations and Matplotlib for visualizations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import eig, svd, matrix_rank, norm
import scipy.linalg as la

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
np.set_printoptions(precision=4, suppress=True)

## Exercise 1: Eigendecomposition and Dynamical Systems (Solved)

In complex systems, eigenvalues and eigenvectors provide crucial information about system stability and behavior. Let's explore this connection with a simple linear dynamical system represented by a matrix transformation.

### Problem Statement

Consider a simple population model with two species that interact. The state of the system at time t is represented by a vector $x(t) = [x_1(t), x_2(t)]^T$, where $x_1(t)$ and $x_2(t)$ are the populations of the two species. 

The system evolves according to a linear difference equation: $x(t+1) = Ax(t)$, where:

$$A = \begin{bmatrix} 0.8 & 0.3 \\ 0.2 & 0.7 \end{bmatrix}$$

1. Find the eigenvalues and eigenvectors of matrix A
2. Using the eigendecomposition, predict the long-term behavior of the system
3. Simulate the system evolution from different initial conditions and visualize the trajectories

In [ ]:
# Define the system matrix
A = np.array([
    [0.8, 0.3],
    [0.2, 0.7]
])

print("System matrix A:")
print(A)

### Step 1: Find eigenvalues and eigenvectors

We use NumPy's eigenvalue decomposition function to find the eigenvalues and eigenvectors.

In [ ]:
# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = eig(A)

print("Eigenvalues:")
print(eigenvalues)
print("\nEigenvectors (by column):")
print(eigenvectors)

# Check our results by verifying that A*v = λ*v for each eigenpair
for i in range(len(eigenvalues)):
    λ = eigenvalues[i]
    v = eigenvectors[:, i]
    Av = A @ v
    λv = λ * v
    print(f"\nFor eigenvalue λ = {λ:.4f}:")
    print(f"A·v = {Av}")
    print(f"λ·v = {λv}")
    print(f"Difference norm: {norm(Av - λv):.8f}")

### Step 2: Interpret the eigenvalues for system behavior

In a discrete-time linear dynamical system x(t+1) = Ax(t):
- Eigenvalues with |λ| < 1 correspond to decaying modes
- Eigenvalues with |λ| > 1 correspond to growing modes
- Eigenvalues with |λ| = 1 correspond to conserved modes

Let's analyze our eigenvalues to understand the long-term behavior of this system.

In [ ]:
# Check magnitudes of eigenvalues
magnitudes = np.abs(eigenvalues)
print("Magnitudes of eigenvalues:", magnitudes)

# Interpret system stability based on eigenvalue magnitudes
if all(magnitudes < 1):
    print("All eigenvalues have magnitude < 1: System is stable, all trajectories converge to zero.")
elif all(magnitudes <= 1) and any(magnitudes == 1):
    print("Some eigenvalues have magnitude = 1: System has conserved modes.")
else:
    print("Some eigenvalues have magnitude > 1: System is unstable, some trajectories grow unbounded.")

Let's also look at how the matrix powers behave, which tells us how the system evolves over multiple time steps.

In [ ]:
# Investigate the behavior of matrix powers
powers = [1, 2, 5, 10, 20, 50]

for power in powers:
    matrix_power = np.linalg.matrix_power(A, power)
    print(f"A^{power} = ")
    print(matrix_power)
    print()

### Step 3: Simulate the system evolution

Now let's simulate and visualize the system evolution from different initial conditions to see how the trajectories behave.

In [ ]:
# Define a function to simulate the system
def simulate_system(A, x0, steps=50):
    x = np.zeros((steps+1, len(x0)))
    x[0] = x0
    
    for t in range(steps):
        x[t+1] = A @ x[t]
    
    return x

# Set up initial conditions and simulation
initial_states = [
    np.array([1.0, 0.0]),    # Only species 1
    np.array([0.0, 1.0]),    # Only species 2
    np.array([1.0, 1.0]),    # Equal populations
    np.array([2.0, 0.5])     # Mixed populations
]

trajectories = [simulate_system(A, x0) for x0 in initial_states]

# Create visualization of trajectories
plt.figure(figsize=(12, 10))

# Plot state space trajectories
plt.subplot(221)
colors = ['b', 'g', 'r', 'c']
labels = ['[1,0]', '[0,1]', '[1,1]', '[2,0.5]']

for i, trajectory in enumerate(trajectories):
    plt.plot(trajectory[:, 0], trajectory[:, 1], 'o-', color=colors[i], alpha=0.6, label=f"x₀ = {labels[i]}")
    plt.plot(trajectory[0, 0], trajectory[0, 1], 'o', color=colors[i], markersize=8)
    
# Plot eigenvectors as direction fields
for i in range(len(eigenvalues)):
    v = eigenvectors[:, i]
    plt.arrow(0, 0, v[0], v[1], color='k', alpha=0.5, width=0.01,
              length_includes_head=True, head_width=0.05, head_length=0.1,
              label=f"eigenvector {i+1}")

plt.grid(True)
plt.xlabel('Species 1 Population')
plt.ylabel('Species 2 Population')
plt.title('State Space Trajectories')
plt.legend()
plt.axis('equal')

# Plot time series of species 1
plt.subplot(222)
for i, trajectory in enumerate(trajectories):
    plt.plot(range(len(trajectory)), trajectory[:, 0], '-', color=colors[i], label=f"x₀ = {labels[i]}")
plt.grid(True)
plt.xlabel('Time Step')
plt.ylabel('Population')
plt.title('Species 1 Population Over Time')
plt.legend()

# Plot time series of species 2
plt.subplot(223)
for i, trajectory in enumerate(trajectories):
    plt.plot(range(len(trajectory)), trajectory[:, 1], '-', color=colors[i], label=f"x₀ = {labels[i]}")
plt.grid(True)
plt.xlabel('Time Step')
plt.ylabel('Population')
plt.title('Species 2 Population Over Time')
plt.legend()

# Plot total population
plt.subplot(224)
for i, trajectory in enumerate(trajectories):
    total_pop = trajectory[:, 0] + trajectory[:, 1]
    plt.plot(range(len(trajectory)), total_pop, '-', color=colors[i], label=f"x₀ = {labels[i]}")
plt.grid(True)
plt.xlabel('Time Step')
plt.ylabel('Population')
plt.title('Total Population Over Time')
plt.legend()

plt.tight_layout()
plt.show()

### Analysis and Conclusions

From our analysis, we can draw several key insights:

1. **Eigenvalue Analysis**: Both eigenvalues have magnitude less than 1, which indicates a stable system where all trajectories converge to a single equilibrium point (in this case, the origin).

2. **Convergence Rate**: The larger eigenvalue (closer to 1) dominates the long-term behavior, determining the slowest rate of convergence.

3. **Trajectory Behavior**: 
   - All trajectories eventually align with the eigenvector corresponding to the dominant eigenvalue
   - The non-dominant mode decays faster, so trajectories first approach this general direction before being drawn along the dominant eigenvector

4. **System Interpretation**: In this population model, regardless of the initial conditions, the populations of both species will eventually decline to zero (as both eigenvalues < 1), but they will do so in a way that maintains a specific ratio between them (determined by the dominant eigenvector).

The eigendecomposition provides powerful insights into the qualitative behavior of this system without needing to simulate every possible scenario.

## Exercise 2: Singular Value Decomposition and Data Analysis (Unsolved)

Singular Value Decomposition (SVD) is a powerful technique with applications in data compression, noise reduction, and uncovering latent patterns in high-dimensional data. In complex systems, SVD can help identify dominant modes or patterns within multivariate data.

### Problem Statement

You are analyzing a dataset of environmental measurements from an ecosystem. The data contains hourly measurements of 5 variables (temperature, humidity, light intensity, CO2 levels, and soil moisture) collected over a 24-hour period. The data matrix X has shape (24, 5) where rows represent time points and columns represent the different variables.

Your tasks:

1. Generate synthetic data with correlated patterns to simulate the environmental measurements
2. Apply SVD to decompose the data matrix
3. Determine how many components are needed to explain at least 90% of the variance
4. Reconstruct the data using only the dominant components and calculate the approximation error
5. Visualize the original variables in the space of the first two singular vectors

Here's some starter code to help you generate the synthetic data:

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data with correlated patterns
hours = np.arange(24)

# Create base patterns (e.g., daily cycles, random fluctuations)
daily_cycle = np.sin(np.pi * hours / 12)  # Peaks at noon
random_fluctuation = np.random.normal(0, 0.2, 24)

# Generate the variables with different weightings of the patterns
temperature = 25 + 5 * daily_cycle + random_fluctuation  # Peaks during the day
humidity = 60 - 20 * daily_cycle + random_fluctuation    # Inverse relationship with temperature
light = 100 * np.maximum(0, daily_cycle) + np.random.normal(0, 5, 24)  # Only positive during day
co2 = 400 + 50 * np.sin(np.pi * (hours - 3) / 12) + random_fluctuation  # Shifted cycle
soil_moisture = 30 - 5 * np.cumsum(daily_cycle) / 4 + random_fluctuation  # Decreases over day

# Create the data matrix
X = np.column_stack([temperature, humidity, light, co2, soil_moisture])

# Display the first few rows
print("Data matrix X shape:", X.shape)
print("First 5 rows:")
print(X[:5])

# Plot the time series data
plt.figure(figsize=(12, 8))
variables = ['Temperature (°C)', 'Humidity (%)', 'Light Intensity (lux)', 'CO2 (ppm)', 'Soil Moisture (%)']

for i, var_name in enumerate(variables):
    plt.subplot(3, 2, i+1)
    plt.plot(hours, X[:, i])
    plt.xlabel('Hour of Day')
    plt.ylabel(var_name)
    plt.grid(True)

plt.tight_layout()
plt.show()

### Your Task

Complete the following steps to analyze this data using SVD:

1. Standardize the data (subtract mean, divide by standard deviation)
2. Apply SVD decomposition to obtain U, Σ, and V^T matrices
3. Calculate how much variance is explained by each singular value
4. Determine the minimum number of components needed to explain 90% of the variance
5. Reconstruct the data using only the dominant components
6. Visualize the original variables in the space of the first two singular vectors
7. Interpret the meaning of the first two principal components in terms of the original variables

Use the code cells below to complete your solution.

In [ ]:
# Step 1: Standardize the data
# Your code here


In [ ]:
# Step 2: Apply SVD decomposition
# Your code here


In [ ]:
# Step 3 & 4: Calculate explained variance and determine number of components
# Your code here


In [ ]:
# Step 5: Reconstruct the data using only dominant components
# Your code here


In [ ]:
# Step 6: Visualize variables in the space of the first two singular vectors
# Your code here


### Hints

1. Use `np.mean()` and `np.std()` with `axis=0` to standardize columns
2. NumPy's `np.linalg.svd(X)` gives you the U, s (singular values), and Vh (V transpose) matrices
3. The explained variance ratio can be calculated as: singular_values^2 / sum(singular_values^2)
4. To reconstruct using k components: X_approx = U[:,:k] @ np.diag(s[:k]) @ Vh[:k,:]
5. Visualize the variables in PC space using the right singular vectors (V)